In [85]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import numpy as np
import warnings

In [2]:
sc = SparkContext(master='local[1]')
spark = SparkSession.builder.appName('Test').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/17 10:16:32 WARN Utils: Your hostname, promo-ds4-gra9-10, resolves to a loopback address: 127.0.1.1; using 51.91.85.76 instead (on interface ens3)
25/11/17 10:16:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/17 10:16:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load Data

In [4]:
df = spark.read.csv("../data/arbresremarquablesparis.csv", sep=";", header=True)

In [6]:
df.show()

25/11/17 10:17:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/17 10:17:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Geo point, idbase, domanialite, arrondissement, complement adresse, numero, adresse, circonference en cm, hauteur en m, stade développement, pépinière, genre, espèce, varieteoucultivar, date de plantation, libellé Français, ID Base, ID arbre, Site, Adresse, Complément d'adresse, Arrondissement, Domanialité, Dénomination usuelle, Dénomination botanique, Autorité taxonomique, Année de plantation, Qualification remarquable, Résumé, Descriptif, Numéro de délibération, Date de la délibération, Label national, Panonceau, Photo 1, Copyright 1
 Schema: Geo point, idbase, domanialite, arrondissement3, complement adresse, numero, adresse6, circonference en cm, hauteur en m, stade développement, pépinière, genre, espèce

+--------------------+---------+-----------+-----------------+------------------+------+--------------------+-------------------+------------+-------------------+---------+------------+--------------------+-----------------+--------------------+--------------------+---------+---------+--------------------+--------------------+--------------------+----------------+-----------------+--------------------+----------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+-----------------------+-------------------+--------------------+--------------------+--------------------+
|           Geo point|   idbase|domanialite|  arrondissement3|complement adresse|numero|            adresse6|circonference en cm|hauteur en m|stade développement|pépinière|       genre|              espèce|varieteoucultivar|  date de plantation|    libellé Français|  ID Base| ID arbre|                Site|           Adresse19|Comp

# Some stats...

In [30]:
age = df.agg({"Année de plantation": "min"}).collect()[0]
print(f"Age de l'arbre le plus ancien : {2025-int(age['min(Année de plantation)'])} ans.")

Age de l'arbre le plus ancien : 423 ans.


In [47]:
# Convert to float
df = df.withColumn("hauteur en m", df["hauteur en m"].astype("float"))
df = df.withColumn("circonference en cm", df["circonference en cm"].astype("float"))

# Get volume of tallest tree
df = df.withColumn("volume", np.pi * ((df["circonference en cm"]/(100 * 2 * np.pi)) ** 2 * df["hauteur en m"]))
df.agg(sf.max_by("volume", "hauteur en m")).show()

+----------------------------+
|max_by(volume, hauteur en m)|
+----------------------------+
|          102.49677806957492|
+----------------------------+



In [65]:
df.groupBy("genre").mean().select(["genre", "avg(hauteur en m)"]).orderBy("avg(hauteur en m)", ascending=False).show()

+--------------+------------------+
|         genre| avg(hauteur en m)|
+--------------+------------------+
|Sequoiadendron|              29.4|
|      Platanus|28.955555555555556|
|      Taxodium|             28.75|
|    Pterocarya|26.285714285714285|
|       Juglans|              26.0|
|         Tilia|              23.0|
|  Liriodendron|             22.75|
|      Aesculus| 21.22222222222222|
|        Ginkgo|            21.125|
|      Fraxinus|              21.0|
|    Calocedrus|              21.0|
|         Pinus|              21.0|
|       Zelkova|             20.25|
|        Betula|              20.0|
|       Sequoia|              20.0|
|         Alnus|              20.0|
|   Metasequoia|              19.0|
|          Acer|              18.8|
|        Cedrus|             18.25|
|       Corylus|              17.5|
+--------------+------------------+
only showing top 20 rows


In [73]:
df.groupby("arrondissement21").count().orderBy("count", ascending=False).show()

25/11/17 11:17:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Arrondissement
 Schema: Arrondissement21
Expected: Arrondissement21 but found: Arrondissement
CSV file: file:///home/duboisj/module-BigData/data/arbresremarquablesparis.csv


+----------------+-----+
|arrondissement21|count|
+----------------+-----+
|              16|   52|
|              12|   35|
|              20|   13|
|              19|   12|
|              18|   10|
|              14|   10|
|               7|    9|
|               5|    7|
|              17|    7|
|              15|    6|
|               8|    6|
|               4|    6|
|              13|    4|
|               3|    3|
|               9|    2|
|             1er|    1|
|              11|    1|
|            NULL|    1|
|              10|    1|
+----------------+-----+



In [ ]:
nb_trees_cimetiere = df.filter(df["adresse6"].contains("CIMETIERE DU PERE LACHAISE")).count()
print(f"Il y a {nb_trees_cimetiere} arbres remarquables au cimetière du père lachaise.\n\n")

Il y a 9 arbres remarquables au cimetière du père lachaise.




25/11/17 11:25:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: adresse
 Schema: adresse6
Expected: adresse6 but found: adresse
CSV file: file:///home/duboisj/module-BigData/data/arbresremarquablesparis.csv
